# Solar energy - Predictions for one MONTH & one STATION

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = 300

import warnings
warnings.filterwarnings("ignore")

Load data and keep only energy efficiency for france

In [2]:
path = "../../../datasets/_classified/kaggle/"
df = pd.read_csv(path + "solar_generation_by_station.csv")

# keeping only values for one country for the predictions


## with a RNN - Recurrent Neural Network

In [3]:
df = pd.read_csv(path + "solar_generation_by_station.csv")
french_stations = [c for c in df.columns if 'FR' in c]
df = df[french_stations]
df.head()

,FR42,FR61,FR72,FR25,FR26,FR52,FR24,FR21,FR83,FR43,FR23,FR10,FR81,FR63,FR41,FR62,FR30,FR51,FR22,FR53,FR82,FR71
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#df.isnull().sum()

# data for 3 years
df = df[-24*365*3:]
df.shape

(26280, 22)

In [5]:
(list(df.columns))

['FR42',
 'FR61',
 'FR72',
 'FR25',
 'FR26',
 'FR52',
 'FR24',
 'FR21',
 'FR83',
 'FR43',
 'FR23',
 'FR10',
 'FR81',
 'FR63',
 'FR41',
 'FR62',
 'FR30',
 'FR51',
 'FR22',
 'FR53',
 'FR82',
 'FR71']

In [6]:
df = df[['FR21', 'FR22', 'FR23', 'FR24', 'FR25']]
df.head()

,FR21,FR22,FR23,FR24,FR25
236688,0.0,0.0,0.0,0.0,0.0
236689,0.0,0.0,0.0,0.0,0.0
236690,0.0,0.0,0.0,0.0,0.0
236691,0.0,0.0,0.0,0.0,0.0
236692,0.0,0.0,0.0,0.0,0.0


In [7]:
df.shape

(26280, 5)

In [8]:
def process_data(data, past):
    """Compute the X and y data.
    X = all the features, of the past (this param can changed)
    y = stations efficiencies of the current day."""
    X = []
    for i in range(len(data)-past-1):
        X.append(data.iloc[i:i+past].values)
    return np.array(X)


lookback = 24 * 2 # two days

y = df['FR23'][lookback+1:] 
X = process_data(df, lookback)
X.shape, y.shape

((26231, 48, 5), (26231,))

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout


def gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=32, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(GRU(units=32, activation='relu', return_sequences=False))
    model.add(Dense(units=1, activation='linear'))
    return model


model = gru_model(X.shape[1:])
model.compile(optimizer='adam', loss='mean_squared_error')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:


X_train = X[:40000]
y_train = y[:40000]
X_test = X[40000:]
y_test = y[40000:]


model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

In [ ]:
len(model_names), len(rmse_train), len(rmse_test)

In [ ]:
plot_scores()

- An RMSE below 0.1 is quite good compared to values ranging from 0 to less than 1.
- Finally only few models are better than the baseline, but not that better.
- Only the linear regression and ridge get better when we deal with months, weeks, days & hours as categorical features with "get dummies" pandas method.
- Facebook lib's prophet is finally not that effective but the forecast of a whole year is probably a difficult goal.

In a new study we'll try to forecast energy efficiency for only a month, which can still be interesting from a business point of view.